# IBM Data Science Capstone Project

## Coursera Course 9

### This notebook contains the code for the execution of the capstone project


In [145]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    beautifulsoup4-4.8.2       |           py36_0         157 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.2-py36_0          conda-forge
    ca-certificates: 2019.11.27-0      --> 2019.11.28-hecc5488_0 conda-forge
    certifi:         2019.11.28-py36_0 --> 2019.11.28-py36_0     conda-f

In [146]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
import json

import requests

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

Libraries imported!


In [2]:
print("Hello Capstone Poject Course")

Hello Capstone Poject Course


In [3]:
pd.__version__

'0.24.1'

#  PART 1: SEGMENTING AND CLUSTERING NEIGHBOURHOODS IN TORONTO
## Scraping the Wikipedia Page
### Using BeautifulSoup

In [150]:
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df_toronto = pd.read_html(str(table))[0]
postcode = df["Postcode"].tolist()
borough = df["Borough"].tolist()
neighbourhood = df["Neighbourhood"].tolist()
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Removing the 'Not assigned' cells in Borough

In [151]:
df_toronto["Borough"] = df_toronto["Borough"].replace("Not assigned", np.nan)
df_toronto.dropna(subset=["Borough"],inplace=True)
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## Combining Neighbourhood's that have the same Postcode seperated by ','

In [152]:
df_toronto_grouped = df_toronto.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ',' .join(x))
df_toronto_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Neighbourhood's with 'Not assigned' are changed to the same name  as the Borough

In [153]:
nan_neigh_rows = df_toronto_grouped.Neighbourhood == 'Not assigned'
df_toronto_grouped.loc[nan_neigh_rows, 'Neighbourhood'] = df_toronto_grouped.loc[nan_neigh_rows, 'Borough']
df_toronto_grouped[nan_neigh_rows]

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,Queen's Park


## Shape of the cleaned DataFrame

In [154]:
df_toronto_grouped.shape

(103, 3)

## Getting coordinates and adding them to the Toronto DataFrame

In [155]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coords = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [156]:
print(coors.shape)
coords.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging the two DataFrames

In [157]:
df_toronto_temp = df_toronto_grouped.set_index('Postcode')
coords_temp = coords.set_index('Postal Code')
df_toronto_coords = pd.concat([df_toronto_temp, coords_temp], axis=1, join='inner')

## Resetting the Index

In [158]:
df_toronto_coords.index.name = 'PostalCode'
df_toronto_coords.reset_index(inplace=True)

## Final DataFrame with added coordinates

In [159]:
print(df_toronto_coords.shape)
df_toronto_coords.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 2: Explore and cluster the neighborhoods in Toronto

In [160]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


## Using Folium to create a Map of Toronto with Boroughs markers on top

In [161]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df_toronto_coords['Latitude'], df_toronto_coords['Longitude'], df_toronto_coords['PostalCode'], df_toronto_coords['Borough'], df_toronto_coords['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Reducing the number of Boroughs to explore

In [162]:
toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_central_df = df_toronto_coords[df_toronto_coords['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print(toronto_central_df.shape)
toronto_central_df.head()

(39, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [163]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, post, borough, neigh in zip(df_toronto_coords['Latitude'], df_toronto_coords['Longitude'], df_toronto_coords['PostalCode'], df_toronto_coords['Borough'], df_toronto_coords['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Using FourSquare API to explore the Boroughs

In [ ]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = ''

In [166]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_central_df['Latitude'], toronto_central_df['Longitude'], toronto_central_df['PostalCode'], toronto_central_df['Borough'], toronto_central_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [167]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighbourhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1698, 9)


,PostalCode,Borough,Neighbourhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


## Check how many venues returned for each postal code

In [168]:
venues_df.groupby(['PostalCode', 'Borough', 'Neighbourhood'])['VenueName'].count()

PostalCode  Borough           Neighbourhood                                                                                       
M4E         East Toronto      The Beaches                                                                                               4
M4K         East Toronto      The Danforth West,Riverdale                                                                              42
M4L         East Toronto      The Beaches West,India Bazaar                                                                            19
M4M         East Toronto      Studio District                                                                                          41
M4N         Central Toronto   Lawrence Park                                                                                             4
M4P         Central Toronto   Davisville North                                                                                          7
M4R         Central Toronto   North Toron

## Analyze venues in each area

In [169]:
# one hot encoding
toronto_central_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_central_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_central_onehot['Borough'] = venues_df['Borough'] 
toronto_central_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_central_onehot.columns[-3:]) + list(toronto_central_onehot.columns[:-3])
toronto_central_onehot = toronto_central_onehot[fixed_columns]

print(toronto_central_onehot.shape)
toronto_central_onehot.head()

(1698, 228)


,PostalCode,Borough,Neighbourhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West,Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Getting the frequency of occurance of each category in an area

In [170]:
toronto_central_venues_freq = toronto_central_onehot.groupby(['PostalCode', 'Borough', 'Neighbourhoods']).mean().reset_index()
print(toronto_central_venues_freq.shape)
toronto_central_venues_freq.head()

(39, 228)


,PostalCode,Borough,Neighbourhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.25000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
1,M4K,East Toronto,"The Danforth West,Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.02381,...,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,0.00000,0.0,0.02381
2,M4L,East Toronto,"The Beaches West,India Bazaar",0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.04878,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.02439,0.0,0.02439
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000


## Getting 10 most occurance venue types in each area

In [171]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighbourhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_central_venues_freq['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_central_venues_freq['Borough']
neighborhoods_venues_sorted['Neighbourhoods'] = toronto_central_venues_freq['Neighbourhoods']

for ind in np.arange(toronto_central_venues_freq.shape[0]):
    row_categories = toronto_central_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
neighborhoods_venues_sorted

,PostalCode,Borough,Neighbourhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Boat or Ferry,Boutique,Harbor / Marina,Airport Food Court,Airport,Bar
31,M6H,West Toronto,"Dovercourt Village,Dufferin",Bakery,Pharmacy,Furniture / Home Store,Brazilian Restaurant,Fast Food Restaurant,Café,Bar,Bank,Supermarket,Middle Eastern Restaurant
32,M6J,West Toronto,"Little Portugal,Trinity",Bar,Asian Restaurant,Restaurant,Coffee Shop,Vietnamese Restaurant,Pizza Place,Men's Store,Café,Yoga Studio,Mexican Restaurant
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Nightclub,Performing Arts Venue,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium
25,M5S,Downtown Toronto,"Harbord,University of Toronto",Café,Bakery,Bookstore,Japanese Restaurant,Restaurant,Bar,College Arts Building,Coffee Shop,Chinese Restaurant,Dessert Shop
34,M6P,West Toronto,"High Park,The Junction South",Café,Bar,Mexican Restaurant,Thai Restaurant,Italian Restaurant,Furniture / Home Store,Fast Food Restaurant,Bookstore,Cajun / Creole Restaurant,Flea Market
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Sandwich Place,Cheese Shop,Pet Store
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Café,Spa,Diner,Sporting Goods Shop,Fast Food Restaurant,Salon / Barbershop,Dessert Shop
19,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Restaurant,Scenic Lookout,Fried Chicken Joint,Brewery,Bar
13,M5A,Downtown Toronto,Harbourfront,Coffee Shop,Bakery,Park,Café,Pub,Restaurant,Mexican Restaurant,Beer Store,Chocolate Shop,Dessert Shop


## Clustering areas

In [172]:
kclusters = 3

toronto_central_venues_freq_clustering = toronto_central_venues_freq.drop(['PostalCode', 'Borough', 'Neighbourhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_central_venues_freq_clustering)

toronto_central_clustered_df = toronto_central_df
toronto_central_clustered_df['Cluster'] = kmeans.labels_

toronto_central_clustered_df = toronto_central_clustered_df.join(neighborhoods_venues_sorted.drop(['Borough', 'Neighbourhoods'], 1).set_index('PostalCode'), on='PostalCode')
toronto_central_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_central_clustered_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,0,Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Boat or Ferry,Boutique,Harbor / Marina,Airport Food Court,Airport,Bar
31,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259,0,Bakery,Pharmacy,Furniture / Home Store,Brazilian Restaurant,Fast Food Restaurant,Café,Bar,Bank,Supermarket,Middle Eastern Restaurant
32,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750,0,Bar,Asian Restaurant,Restaurant,Coffee Shop,Vietnamese Restaurant,Pizza Place,Men's Store,Café,Yoga Studio,Mexican Restaurant
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.636847,-79.428191,0,Breakfast Spot,Café,Coffee Shop,Nightclub,Performing Arts Venue,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium
25,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,0,Café,Bakery,Bookstore,Japanese Restaurant,Restaurant,Bar,College Arts Building,Coffee Shop,Chinese Restaurant,Dessert Shop
34,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,0,Café,Bar,Mexican Restaurant,Thai Restaurant,Italian Restaurant,Furniture / Home Store,Fast Food Restaurant,Bookstore,Cajun / Creole Restaurant,Flea Market
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Sandwich Place,Cheese Shop,Pet Store
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Café,Spa,Diner,Sporting Goods Shop,Fast Food Restaurant,Salon / Barbershop,Dessert Shop
19,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Italian Restaurant,Hotel,Café,Restaurant,Scenic Lookout,Fried Chicken Joint,Brewery,Bar
13,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Café,Pub,Restaurant,Mexican Restaurant,Beer Store,Chocolate Shop,Dessert Shop


## Showing those clusters onto a map

In [173]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_central_clustered_df['Latitude'], toronto_central_clustered_df['Longitude'], toronto_central_clustered_df['PostalCode'], toronto_central_clustered_df['Borough'], toronto_central_clustered_df['Neighbourhood'], toronto_central_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters